In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

### Load the data

In [2]:
data = pd.read_csv("../data/alzheimers_disease_data.csv")
data = data.drop("DoctorInCharge", axis=1) # this attribute is confidential in the data, and thus not useful 
data.head()

,PatientID,Age,Gender,Ethnicity,EducationLevel,BMI,Smoking,AlcoholConsumption,PhysicalActivity,DietQuality,...,FunctionalAssessment,MemoryComplaints,BehavioralProblems,ADL,Confusion,Disorientation,PersonalityChanges,DifficultyCompletingTasks,Forgetfulness,Diagnosis
0,4751,73,0,0,2,22.927749,0,13.297218,6.327112,1.347214,...,6.518877,0,0,1.725883,0,0,0,1,0,0
1,4752,89,0,0,0,26.827681,0,4.542524,7.619885,0.518767,...,7.118696,0,0,2.592424,0,0,0,0,1,0
2,4753,73,0,3,1,17.795882,0,19.555085,7.844988,1.826335,...,5.895077,0,0,7.119548,0,1,0,1,0,0
3,4754,74,1,0,1,33.800817,1,12.209266,8.428001,7.435604,...,8.965106,0,1,6.481226,0,0,0,0,0,0
4,4755,89,0,0,0,20.716974,0,18.454356,6.310461,0.795498,...,6.045039,0,0,0.014691,0,0,1,1,0,0


### Oversampling by ethnicity

In [3]:
e = data["Ethnicity"]
print('Original dataset shape %s' % Counter(e))

ethnicity_counts = dict(data["Ethnicity"].value_counts())
num_ethnicities = len(ethnicity_counts)
max_count = max(ethnicity_counts.values())

strategy_over = {ethnicity: max_count for ethnicity in range(num_ethnicities)}
over = SMOTE(sampling_strategy=strategy_over)
print(strategy_over, sep='\n')

data_over, e_over=over.fit_resample(data, e)
print('Oversampled dataset shape %s' % Counter(e_over))

Original dataset shape Counter({0: 1278, 1: 454, 3: 211, 2: 206})
{0: 1278, 1: 1278, 2: 1278, 3: 1278}
Oversampled dataset shape Counter({0: 1278, 3: 1278, 1: 1278, 2: 1278})


### Split the data

In [4]:
X, y = data_over.drop("Diagnosis", axis=1), data_over["Diagnosis"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=22, stratify=y)
X_train.head()

,PatientID,Age,Gender,Ethnicity,EducationLevel,BMI,Smoking,AlcoholConsumption,PhysicalActivity,DietQuality,...,MMSE,FunctionalAssessment,MemoryComplaints,BehavioralProblems,ADL,Confusion,Disorientation,PersonalityChanges,DifficultyCompletingTasks,Forgetfulness
2743,6857,63,1,1,1,20.379268,0,1.511398,3.982210,3.736809,...,2.413633,5.352684,0,0,1.506161,0,0,0,0,0
3313,6844,76,0,2,1,34.363994,0,14.900643,7.481645,8.663097,...,2.726072,4.301770,0,0,5.130438,0,0,0,0,0
1793,6544,66,0,3,1,18.060797,0,16.273489,1.683292,2.451237,...,21.552252,1.817850,0,0,7.586159,0,1,0,0,0
1578,6329,65,1,0,1,15.012071,0,2.415020,1.297114,3.839334,...,12.625008,5.192733,1,0,6.845832,1,0,0,0,0
4991,6809,75,0,3,2,33.446922,0,7.315071,6.168820,7.012029,...,18.053187,6.108258,0,0,6.860218,0,0,0,0,0


### Train the model

In [5]:
classifier = MLPClassifier(max_iter=5000, random_state=12)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

In [6]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      1.00      0.89       992
           1       0.90      0.12      0.22       286

    accuracy                           0.80      1278
   macro avg       0.85      0.56      0.55      1278
weighted avg       0.82      0.80      0.74      1278



In [7]:
X["Ethnicity"].value_counts()

Ethnicity
0    1278
3    1278
1    1278
2    1278
Name: count, dtype: int64

### Compare performance based on ethnicity

In [8]:
# Join X_test, y_test, y_pred for analysis
results = X_test.copy()
results["TrueDiagnosis"] = y_test
results["PredictedDiagnosis"] = y_pred

In [9]:
# Ethnicity 0
eth0_results = results[results["Ethnicity"] == 0]
print(classification_report(eth0_results["TrueDiagnosis"], eth0_results["PredictedDiagnosis"]))

              precision    recall  f1-score   support

           0       0.71      0.99      0.83       211
           1       0.91      0.20      0.33       106

    accuracy                           0.73       317
   macro avg       0.81      0.59      0.58       317
weighted avg       0.78      0.73      0.66       317



In [10]:
# Ethncity 1
eth1_results = results[results["Ethnicity"] == 1]
print(classification_report(eth1_results["TrueDiagnosis"], eth1_results["PredictedDiagnosis"]))

              precision    recall  f1-score   support

           0       0.80      0.99      0.89       244
           1       0.82      0.13      0.23        68

    accuracy                           0.80       312
   macro avg       0.81      0.56      0.56       312
weighted avg       0.81      0.80      0.74       312



In [11]:
# Ethnicity 2  
eth2_results = results[results["Ethnicity"] == 2]
print(classification_report(eth2_results["TrueDiagnosis"], eth2_results["PredictedDiagnosis"]))

              precision    recall  f1-score   support

           0       0.79      1.00      0.88       244
           1       1.00      0.04      0.08        69

    accuracy                           0.79       313
   macro avg       0.89      0.52      0.48       313
weighted avg       0.83      0.79      0.71       313



In [12]:
# Ethncity 3
eth3_results = results[results["Ethnicity"] == 3]
print(classification_report(eth3_results["TrueDiagnosis"], eth3_results["PredictedDiagnosis"]))

              precision    recall  f1-score   support

           0       0.88      1.00      0.93       293
           1       1.00      0.05      0.09        43

    accuracy                           0.88       336
   macro avg       0.94      0.52      0.51       336
weighted avg       0.89      0.88      0.83       336



In [13]:
from fairness_metrics import demographic_parity, equal_opportunity, disparate_impact

# DI >= 0.8 is a pre-established threshold for fairness
# DP and EO need to be as close to 0 as possible

dp = demographic_parity(results, 0, 1)
eo = equal_opportunity(results, 0, 1)
di = disparate_impact(results, 0, 1)

print(dp, eo, di)

from fairness_metrics import demographic_parity, equal_opportunity, disparate_impact

# DI >= 0.8 is a pre-established threshold for fairness
# DP and EO need to be as close to 0 as possible

dp = demographic_parity(results, 0, 2)
eo = equal_opportunity(results, 0, 2)
di = disparate_impact(results, 0, 2)

print(dp, eo, di)


from fairness_metrics import demographic_parity, equal_opportunity, disparate_impact

# DI >= 0.8 is a pre-established threshold for fairness
# DP and EO need to be as close to 0 as possible

dp = demographic_parity(results, 0, 3)
eo = equal_opportunity(results, 0, 3)
di = disparate_impact(results, 0, 3)

print(dp, eo, di)

0.03729879479090836 0.09486166007905128 2.057929452251219
0.0629705405105774 0.08695652173913049 7.569926393270243
0.06660282409493767 0.08695652173913049 12.18927444794953
